In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,7730
2,Huelva,Confirmados PDIA 14 días,2556
3,Huelva,Tasa PDIA 14 días,"500,44738743360165"
4,Huelva,Confirmados PDIA 7 días,1279
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,4
629,Municipio de Huelva sin especificar,Total Confirmados,105
630,Municipio de Huelva sin especificar,Curados,27


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  7730.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2115.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 426 personas en los últimos 7 días 

Un positivo PCR cada 194 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,7730.0,1279.0,2556.0,510743.0,250.419487,500.447387,79.0
Huelva-Costa,4081.0,724.0,1481.0,288115.0,251.288548,514.030856,39.0
Condado-Campiña,2415.0,389.0,705.0,155057.0,250.875484,454.671508,24.0
Huelva (capital),2115.0,337.0,739.0,143663.0,234.576753,514.398279,22.0
Sierra de Huelva-Andévalo Central,1130.0,162.0,358.0,67571.0,239.747821,529.813085,18.0
Moguer,195.0,65.0,94.0,22088.0,294.277436,425.570445,10.0
Lepe,633.0,118.0,230.0,27431.0,430.170245,838.467427,7.0
Corteconcepción,11.0,8.0,8.0,548.0,1459.854015,1459.854015,5.0
Punta Umbría,177.0,40.0,76.0,15242.0,262.432752,498.622228,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Aracena,275.0,30.0,121.0,8107.0,370.050574,1492.537313,2.0
Corteconcepción,11.0,8.0,8.0,548.0,1459.854015,1459.854015,5.0
Villablanca,49.0,15.0,36.0,2848.0,526.685393,1264.044944,0.0
Almendro (El),16.0,6.0,10.0,826.0,726.392252,1210.653753,0.0
Palma del Condado (La),353.0,57.0,105.0,10761.0,529.690549,975.745749,2.0
Campillo (El),49.0,10.0,19.0,2023.0,494.315373,939.199209,2.0
Villarrasa,86.0,17.0,20.0,2176.0,781.250000,919.117647,1.0
Lepe,633.0,118.0,230.0,27431.0,430.170245,838.467427,7.0
Marines (Los),5.0,3.0,3.0,396.0,757.575758,757.575758,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aracena,275.0,30.0,121.0,8107.0,370.050574,1492.537313,2.0,0.247934
San Silvestre de Guzmán,7.0,1.0,4.0,623.0,160.513644,642.054575,0.0,0.250000
Manzanilla,44.0,2.0,8.0,2135.0,93.676815,374.707260,0.0,0.250000
Villalba del Alcor,81.0,3.0,12.0,3338.0,89.874176,359.496705,0.0,0.250000
Gibraleón,315.0,22.0,88.0,12607.0,174.506227,698.024907,0.0,0.250000
Escacena del Campo,88.0,3.0,10.0,2284.0,131.348511,437.828371,0.0,0.300000
Beas,33.0,4.0,13.0,4257.0,93.962885,305.379375,0.0,0.307692
Zalamea la Real,39.0,2.0,6.0,3068.0,65.189048,195.567145,0.0,0.333333
Zarza-Perrunal (La),12.0,2.0,6.0,1267.0,157.853197,473.559590,0.0,0.333333
